---

# Python dependences

In [2]:
!pip install pymdptoolbox

---

# EXERCISE 1: What to do at the airport?

You are travelling and have some time to kill at the aiport. There are three things you could spend your time doing:
  
1) You could have a coffee.

This has a probability of $0.8$ of giving you time to relax with a tasty beverage, and a utility of $10$.
It also has a probability of $0.2$ of providing you with a nasty cup from over-roasted beans that annoys you,
and outcome with a utility of $-5$.

2) You could shop for clothes.

This has a probability of $0.1$ that you will find a great outfit at a good price, utility $20$. However, it
has a probability of $0.9$ that you end up wasting money on over-priced junk, utility $-10$.

3) You could have a bite to eat.

This has a probability of $0.8$ that you find something rather mediocre that prevents you from being too hungry
during your flight, utility $2$, and a probability of $0.2$ that you find something filling and tasty, utility $5$.

> __QUESTION 1(a):__ What should you do if you take the principle of maximum expected utility to be your decision criterion?

> __QUESTION 1(b):__ What should you do if you take the principle of maximax decision criterion to be your decision criterion?

> __QUESTION 1(c):__ What should you do if you take the principle of maximin decision criterion to be your decision criterion?
    

In [3]:
import numpy as np

# ----- INITIALISING DATA OF THE PROBLEM -----
# choices
actions = ('coffee', 'clothes', 'eat')

# utility of each state
U = np.array([[10, -5],  # utility of good beverage or annoying beverage, choosing coffee
              [20, -10], # utility of great outfit or wasting money, choosing clothes
              [2, 5]])   # utility of mediocre meal or tasty meal, choosing eat

# probability of each state
P = np.array([[0.8, 0.2],  # probability of good beverage or annoying beverage, choosing coffee
              [0.1, 0.9],  # probability of great outfit or wasting money, choosing clothes
              [0.8, 0.2]]) # probability of mediocre meal or tasty meal, choosing eat

# calculating Expected Utility for actions
EU = U*P
print('expected utilities:\n', EU, '\n')

# ----- MAXIMUM EXPECTED UTILITY - MEU -----
# calculating utility for every choice
EU_choice = EU.sum(axis=1)

# calculating action with the maximum expected utility
MEU_action = actions[np.argmax(EU_choice)]
print('MEU:', MEU_action)

# ----- MAXIMAX UTILITY - MXX -----
# calculating action with the maximax utility
MXX_action = actions[np.argmax(np.max(U, axis=1))]
print('MaxiMax:', MXX_action)

# ----- MAXIMIN UTILITY - MXN -----
# calculating action with the maximin utility
MXN_action = actions[np.argmax(np.min(U, axis=1))]
print('MaxiMax:', MXN_action)

expected utilities:
 [[ 8.  -1. ]
 [ 2.  -9. ]
 [ 1.6  1. ]] 

MEU: coffee
MaxiMax: clothes
MaxiMax: eat


---

# EXERCISE 2: Solving a MDP with MDP toolbox

We have four states and four actions.

The actions are: 0 is Right, 1 is Left, 2 is Up and 3 is Down.

The states are 0, 1, 2, 3, and they are arranged like this:
    
$$
\begin{array}{cc}
2 & 3\\
0 & 1\\
\end{array}
$$

The motion model provides:
*   0.8 probability of moving in the direction of the action,
*   0.1 probability of moving in each of the directions perpendicular to that of the action.

So that 2 is Up from 0 and 1 is Right of 0, and so on. The cost of any action (in any state) is -0.04.

In case of "infeasible" movements, the agent remains in the current state with probability 0.8+0.1. Consider the perpendicular directions in any case.

The reward for state 3 is 1, and the reward for state 1 is -1.

The agent does not leave those state 3 and state 1. Keep attention when modeling the transition.

Set discount factor equal to 0.99.

> __QUESTION 2(a):__ What is the policy based on the Value iteration algorithm?

> __QUESTION 2(b):__ What is the policy based on the Policy iteration algorithm?

> __QUESTION 2(c):__ What is the policy based on the Q-Learning algorithm?

> __QUESTION 2(d):__ Look at the **setVerbose**() function and the time attribute of the MDP objects in MDPToolbox and use them to compare the number of iterations (hint: see the iter attribute) and the CPU time used to come up with a solution (hint: see the time attribute) in the Value iteration algorithm and Policy iteration algorithm resolutions.


In [4]:
import numpy as np
import mdptoolbox

# ----- INITIALISING DATA OF THE PROBLEM -----
# verbose
verbose = 0

# TRANSITION MODEL p(final_state | action, initial_state)
P = np.array([[[0.1, 0.8, 0.1, 0],   # final_state | right, state0
               [0,   1,   0,   0],   # final_state | right, state1
               [0.1, 0,   0.1, 0.8], # final_state | right, state2
               [0,   0,   0,   1]],  # final_state | right, state3

              [[0.9, 0,   0.1, 0],  # final_state | left, state0
               [0,   1,   0,   0],  # final_state | left, state1
               [0.1, 0,   0.9, 0],  # final_state | left, state2
               [0,   0,   0,   1]], # final_state | left, state3

              [[0.1, 0.1, 0.8, 0],   # final_state | up, state0
               [0,   1,   0,   0],   # final_state | up, state1
               [0,   0,   0.9, 0.1], # final_state | up, state2
               [0,   0,   0,   1]],  # final_state | up, state3

              [[0.9, 0.1, 0,   0],   # final_state | down, state0
               [0,   1,   0,   0],   # final_state | down, state1
               [0.8, 0,   0.1, 0.1], # final_state | down, state2
               [0,   0,   0,   1]]]) # final_state | down, state3

# REWARD FUNCTION r(initial_state, action)
R = np.array([[-0.04, -0.04, -0.04, -0.04], # reward from state0
              [-1,    -1,    -1,    -1],    # reward from state1
              [-0.04, -0.04, -0.04, -0.04], # reward from state2
              [1,     1,     1,     1]])    # reward from state3

# CHECK DATA
mdptoolbox.util.check(P, R)

# ----- VALUE ITERATION ALGORITHM -----
vi = mdptoolbox.mdp.ValueIteration(P, R, 0.99)
if verbose: vi.setVerbose()
vi.run()
print()

print('Utility values:', vi.V)
print('Iterations:', vi.iter)
print('Policy:', vi.policy)

# ----- POLICY ITERATION ALGORITHM -----
pi = mdptoolbox.mdp.PolicyIteration(P, R, 0.99)
if verbose: pi.setVerbose()
pi.run()
print()

print('Utility values:', pi.V)
print('Iterations:', pi.iter)
print('Policy:', pi.policy)

# ----- Q-LEARNING ALGORITHM -----
qi = mdptoolbox.mdp.QLearning(P, R, 0.99)
if verbose: qi.setVerbose()
qi.run()
print()

print('Utility values:', qi.V)
print('Policy:', qi.policy)


Utility values: (88.23133912867954, -99.9949804281095, 97.54814303782808, 99.9949804281095)
Iterations: 985
Policy: (1, 0, 0, 0)

Utility values: (88.23635870057004, -99.99999999999991, 97.5531626097185, 99.99999999999991)
Iterations: 3
Policy: (1, 0, 0, 0)

Utility values: (1.3217831269597031, -19.63078227794021, 19.350445427776574, 61.500930791081785)
Policy: (1, 2, 0, 0)
